# Телеграм бот с ИИ, анализирующий данные и визуализирующий их для пользователя

Цель работы: Проанализировать данные о продажах при помощи ии, визуализировать их для пользователя и внедрить это в телеграм бота.

### План работы

1. Открыть файл с данными, изучить общую информацию
2. Провести предобработку данных (проверка на пропуски, дубли, типы данных в столбцах)
3. Прописать промпты для g4f и нормализовать его работу с данными для анализа.
4. Подготовить функции создания графиков для дальнейших их запусков
5. Написать телеграм бота который будет принимать запросы, отправлять их ИИ и позже присылать ответ с png и html созданного графика. 

Для начала импортируем нужные библиотеки

In [1]:
import re
import pandas as pd
import plotly.express as px
from g4f.client import Client
from ydata_profiling import ProfileReport
import telebot
import dataframe_image as dfi
from telebot import types

После указания библиотек нужно импортировать файл с которым мы будем работать. Сразу же проверим целостность его данных, их типы и количество.

In [2]:
sales = pd.read_csv('ml_1. Прогнозирование спроса на товары в розничной торговле на основе данных о продажах и маркетинговой активности.csv')
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Дата                      1000 non-null   object 
 1   Товар                     1000 non-null   object 
 2   Категория товара          1000 non-null   object 
 3   Количество продаж         1000 non-null   int64  
 4   Цена                      1000 non-null   float64
 5   Скидки                    1000 non-null   object 
 6   Маркетинговая активность  1000 non-null   object 
 7   Погода                    1000 non-null   object 
 8   Демографические данные    1000 non-null   object 
 9   Рейтинг товара            1000 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 78.3+ KB


В нашем файле отсутствуют пустующие столбцы, а также типы данных на первый взгляд логичны. Из этого делаем вывод, что удалять или форматировать ничего не нужно, поскольку целостность данных позволяет провести верный анализ.

Далее бегло осмотрим взятый датафрейм:

In [3]:
sales.head(5)

,Дата,Товар,Категория товара,Количество продаж,Цена,Скидки,Маркетинговая активность,Погода,Демографические данные,Рейтинг товара
0,2024-05-22,Product0,Косметика,100,215.90,30%,Email-рассылка,Снег,"18-25 лет, мужчины",5.0
1,2024-05-22,Product1,Косметика,8,160.93,30%,Социальные сети,Снег,"25-40 лет, женщины",3.8
2,2024-05-22,Product2,Продукты питания,37,290.97,20%,Скидочные купоны,Снег,Семейные покупатели,4.5
3,2024-05-22,Product3,Одежда,30,722.79,10%,Скидочные купоны,Снег,40+,5.0
4,2024-05-22,Product4,Электроника,80,297.89,20%,Скидочные купоны,Солнечно,"18-25 лет, мужчины",5.0


Судя по имеющимся данным мы можем проанализировать количество продаж в соотношении каких-либо факторов.

Перед началом работы детальнее рассмотрим потенциально ненужный столбец:

In [4]:
print(sales['Дата'].unique()) # Уникальные значения в столбце

['2024-05-22']


По выводу кода мы видим, что уникальное значение всего одно. Смело избавляемся от данного столбца и упрощаем себе работу:

In [5]:
sales = sales.drop(labels='Дата', axis=1)
sales

,Товар,Категория товара,Количество продаж,Цена,Скидки,Маркетинговая активность,Погода,Демографические данные,Рейтинг товара
0,Product0,Косметика,100,215.90,30%,Email-рассылка,Снег,"18-25 лет, мужчины",5.0
1,Product1,Косметика,8,160.93,30%,Социальные сети,Снег,"25-40 лет, женщины",3.8
2,Product2,Продукты питания,37,290.97,20%,Скидочные купоны,Снег,Семейные покупатели,4.5
3,Product3,Одежда,30,722.79,10%,Скидочные купоны,Снег,40+,5.0
4,Product4,Электроника,80,297.89,20%,Скидочные купоны,Солнечно,"18-25 лет, мужчины",5.0
...,...,...,...,...,...,...,...,...,...
995,Product995,Одежда,36,998.41,10%,Скидочные купоны,Снег,Семейные покупатели,5.0
996,Product996,Одежда,22,962.52,30%,Социальные сети,Пасмурно,"25-40 лет, женщины",5.0
997,Product997,Продукты питания,50,11.42,30%,Email-рассылка,Солнечно,"18-25 лет, мужчины",3.8
998,Product998,Электроника,47,72.21,Бесплатная доставка,Рекламная кампания,Снег,"25-40 лет, женщины",4.2


Наименования товаров и демографические данные нам ещё будут нужны, так что трогать их не станем. 

Пока интереса ради глянем какие мы имеем категории:

In [6]:
print('Категории товаров:')
[print(i) for i in sales['Категория товара'].unique()]

Категории товаров:
Косметика
Продукты питания
Одежда
Электроника


[None, None, None, None]

Хоть у нас всего и 4 категории товара, мы не станем усложнять жизнь ни себе ни нашему серверу/компьютеру и раздробим наш датафрейм под каждую категорию.

Мы можем сделать это циклом, но куда проще для нашей цели будет использовать встроенные функции pandas:

In [7]:
grouped = sales.groupby('Категория товара')
categories = {category: group.reset_index(drop=True) for category, group in grouped} # создаём словарь с ключом имени категории и значением датафрейм

for category, group_df in categories.items():
    print(f"\nDataFrame для категории {category}:")
    print(group_df.head(5))


DataFrame для категории Косметика:
      Товар Категория товара  Количество продаж    Цена Скидки  \
0  Product0        Косметика                100  215.90    30%   
1  Product1        Косметика                  8  160.93    30%   
2  Product5        Косметика                 18  667.44    20%   
3  Product7        Косметика                 66  468.02    10%   
4  Product8        Косметика                 72  317.71    30%   

  Маркетинговая активность    Погода Демографические данные  Рейтинг товара  
0           Email-рассылка      Снег     18-25 лет, мужчины             5.0  
1          Социальные сети      Снег     25-40 лет, женщины             3.8  
2       Рекламная кампания  Солнечно     18-25 лет, мужчины             4.5  
3           Email-рассылка      Снег     25-40 лет, женщины             4.2  
4          Социальные сети  Солнечно    Семейные покупатели             3.8  

DataFrame для категории Одежда:
       Товар Категория товара  Количество продаж    Цена          

Мы получили датафреймы, но в них остались бесполезные столбцы "Категория товара". Избавимся от этих столбцов:

In [8]:
for category, group_df in categories.items():
    categories[category] = categories[category].drop(labels='Категория товара', axis=1)

for category, group_df in categories.items():
    print(f"\nDataFrame для категории {category}:")
    print(group_df)


DataFrame для категории Косметика:
          Товар  Количество продаж    Цена               Скидки  \
0      Product0                100  215.90                  30%   
1      Product1                  8  160.93                  30%   
2      Product5                 18  667.44                  20%   
3      Product7                 66  468.02                  10%   
4      Product8                 72  317.71                  30%   
..          ...                ...     ...                  ...   
254  Product972                 57  495.93  Бесплатная доставка   
255  Product974                 28  528.75                  30%   
256  Product978                 81  593.51                  20%   
257  Product991                 50  451.25                  30%   
258  Product992                 75  935.88                  30%   

    Маркетинговая активность    Погода Демографические данные  Рейтинг товара  
0             Email-рассылка      Снег     18-25 лет, мужчины             5.0  

На данном этапе мы имеем датафреймы готовые к работе с plotly и g4f.

# Дальше ведутся работы

отчёт о данных pandas profile

In [9]:
profile = ProfileReport(sales, title="Pandas Profiling Report")

# запускаем показ профиля
# profile

Заранее создаём функцию для анализа данных с помощью ИИ

In [10]:
def g4f_analyze(variant):
    client = Client()
    prompt = 'Тебе даётся датафрейм продаж. Проведи анализ и сделай вывод, \
    опиши всё словами. Бери в учёт все столбцы.'
    content = ''
    response = client.chat.completions.create(
        model="gpt-4o",
        prompt=prompt,
        messages=[{"role": "user", "content": content}],
    )
    return response.choices[0].message.content

В качестве бонуса добавим функцию для ответа на вопросы

In [11]:
def g4f_question(content:str, model='gpt-4o'):
    prompt = '''Ты искуственный интеллект для анализа данных датафрейма.
Ты должен отвечать только на русском языке в официальном стиле!

Твой функционал включает в себя:
1. Анализ данных.
2. Вывод результата анализа текстом без советов по написанию кода для этого анализа.
3. Вывод графиков по датафрейму в формате png или html.
4. Ответ пользователю на интересующие его вопросы по поводу твоего функционала, или другим темам..
'''
    i = 1
    while i <= 10:
        print(f'Попытка №{i}')
        client = Client()
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": f'{prompt}'
                },
                {
                    "role": "user",
                    "content": f'{content}'
                }
            ]
        )
        res = response.choices[0].message.content
        i += 1
        clear_text = re.sub(r"[A-Za-z0-9]", "", res)
        if len(clear_text) > 25:
            if 'model' not in res.lower() and 'blackbox' not in res.lower():
                return res.strip()
    return 'Повторите попытку снова'

Создание Фотографий и Html для показа пользователю:

In [12]:
photo = sales.head(5)
sales.to_html("main_df.html", index=True)
dfi.export(photo,"main_df.png")

## Телеграм бот

In [ ]:
TOKEN = '7677363072:AAGCj1dhXNqaH-j56lEgW3BJ1HoJx8W4kcw' # сюда вставить свой токен TG бота
bot = telebot.TeleBot(TOKEN)
user_states = {}

@bot.message_handler(commands=['start'])
def start(message):
    question = False
    
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Справка о боте")
    btn2 = types.KeyboardButton("Показать таблицу")
    btn3 = types.KeyboardButton("Начать работу с данными")
    btn4 = types.KeyboardButton("Задать вопрос ИИ")
    markup.add(btn1, btn2, btn3, btn4)
    
    txt = f"Привет, {message.from_user.first_name}! Я бот для анализа данных датафреймов, просмотрите мой функционал в командах."
    bot.send_message(message.chat.id, text=txt, reply_markup=markup)
    
    user_states[message.chat.id] = 'MAIN_MENU'

@bot.message_handler(func=lambda message: user_states.get(message.chat.id) == 'MAIN_MENU')
def check_buttons(message):
    if message.text == 'Справка о боте':
        txt = '''   Я рад, что вы решили прочитать справку перед началом работы!

    Мой функционал включает в себя анализ и рисование графиков по тем данным что я имею.
К сожалению я являюсь демо-версией более гибкого в использовании бота, который должен принимать в себя любые данные
в формате excel или csv. Мой способ анализа основан на использовании g4f, что сильно ограничивает меня в работе,
потому в будущем я хотел бы иметь платную версию gpt для анализа более крупных датафреймов/датасетов.

Основные функции:
    1. Вывод датафреймов имеющихся в наличии в формате png и html
    2. Анализ выбранного датафрейма и вывод его графика в формате html или png(по выбору)
    3. Задать вопрос ИИ

Примечание:
    Анализ данных может занимать более 3 минут, а ответ вовсе вам не понравиться, поэтому, пожалуйста, будьте терпеливы и помните, что я могу сломаться
из-за частых запросов. Обязательно обождите обработки вашего запроса перед созданием нового!'''
        bot.send_message(message.chat.id, txt)

    elif message.text == 'Показать таблицу':
        bot.send_message(message.chat.id, "Отправляю вам обрезанный датафрейм в формате png и полный в формате html:")
        bot.send_photo(message.chat.id, open('main_df.png','rb'))
        bot.send_document(message.chat.id, open('main_df.html', 'rb'))

    elif message.text == 'Начать работу с данными':
        bot.send_message(message.chat.id, 'Начата работа с данными')

    elif message.text == 'Задать вопрос ИИ':
        user_states[message.chat.id] = 'QUESTIONS_PROCESSING'

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        back = types.KeyboardButton("Вернуться в главное меню")
        markup.add(back)
        bot.send_message(message.chat.id, 'Я готов отвечать на ваши вопросы!', reply_markup=markup)

@bot.message_handler(func=lambda message: user_states.get(message.chat.id) == 'QUESTIONS_PROCESSING')
def questions_processing(message):
    if message.text == "Вернуться в главное меню":
        back_to_main_menu(message)
    else:
        bot.send_message(message.chat.id, text=g4f_question(message.text))

def back_to_main_menu(message):
    user_states[message.chat.id] = 'MAIN_MENU'
    start(message)


if __name__ == '__main__':
    bot.polling(none_stop=True)

In [ ]:
# px.scatter(categories['Косметика'], x='Цена', y='Количество продаж', color='Погода',)